In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import multiprocessing as mp
from MD_Parser import *
from kern_help import *
import math
import pickle

It is expensive to compute the covariance matrix K. This notebook examines how K can be efficiently computed with parallel processors.

### Get example data.

In [2]:
outfile ='/Users/jonpvandermause/Research/GP/Datasets/SiC_MD/sic_md.out'
Si_MD_Parsed = parse_qe_pwscf_md_output(outfile)

# set crystal structure
dim = 3
alat = 4.344404578
unit_cell = [[0.0, alat/2, alat/2], [alat/2, 0.0, alat/2], \
                    [alat/2, alat/2, 0.0]] # fcc primitive cell
unit_pos = [['Si',[0,0,0]],['Si',[alat/4, alat/4, alat/4]]]
brav_mat = np.array([[0.0, alat/2, alat/2], [alat/2, 0.0, alat/2], \
                    [alat/2, alat/2, 0.0]])*dim
brav_inv = np.linalg.inv(brav_mat)

# bravais vectors
vec1 = brav_mat[:,0].reshape(3,1)
vec2 = brav_mat[:,1].reshape(3,1)
vec3 = brav_mat[:,2].reshape(3,1)

# build force field from single snapshot
cutoff = 3.6
pos = Si_MD_Parsed[1]['positions']
typs = Si_MD_Parsed[1]['elements']
fcs = fc_conv(Si_MD_Parsed[2]['forces'])

### Test covariance functions.

In [3]:
noa = len(pos)
time1 = time.time()
envs = get_envs(pos, typs, brav_mat, brav_inv, vec1, vec2, vec3, cutoff)
time2 = time.time()
print('it takes %.2f s to create %i atomic environments, or about %.2f s per environment.' % (time2-time1,noa,(time2-time1)/noa))

it takes 3.37 s to create 54 atomic environments, or about 0.06 s per environment.


Check how long it takes to compare two atomic environments.

In [4]:
x1 = envs[0]
x2 = envs[1]
d1 = 'xrel'
d2 = 'yrel'
sig = 1
ls = 1

time1 = time.time()
two_body(x1, x2, d1, d2, sig, ls)
time2 = time.time()
print('it takes %.3f us to calculate the two body kernel between two chemical environments.\n\
 the first chemical environment has %i atoms.\n\
 the second chemical environment has %i atoms.\n\
 there are %i possible distances compared, so it takes about %.3f us per distance.'
      % ((time2-time1)*1e6,len(x1['dists']),len(x2['dists']),len(x1['dists'])*len(x2['dists']),\
        1e6*(time2-time1)/(len(x1['dists'])*len(x2['dists']))))

it takes 328.064 us to calculate the two body kernel between two chemical environments.
 the first chemical environment has 23 atoms.
 the second chemical environment has 24 atoms.
 there are 552 possible distances compared, so it takes about 0.594 us per distance.


In [5]:
# get number of triplets in chemical environment
def count_trips(x1):
    count = 0
    for n in x1['trip_dict']['dists']:
        count+=len(n)
    return count

In [6]:
time1 = time.time()
three_body(x1, x2, d1, d2, sig, ls)
time2 = time.time()
print('it takes %.3f s to calculate the three body kernel between two chemical environments.\n\
 the first chemical environment has %i atoms and %i triplets.\n\
 the second chemical environment has %i atoms and %i triplets.\n\
 there are %i possible triplet comparisons, so it takes about %.3f us per triplet.'
      % ((time2-time1),len(x1['dists']),count_trips(x1),len(x2['dists']),count_trips(x2),\
        count_trips(x1)*count_trips(x2),1e6*(time2-time1)/(count_trips(x1)*count_trips(x2))))

it takes 0.124 s to calculate the three body kernel between two chemical environments.
 the first chemical environment has 23 atoms and 276 triplets.
 the second chemical environment has 24 atoms and 300 triplets.
 there are 82800 possible triplet comparisons, so it takes about 1.494 us per triplet.


Test covariance row function.

In [16]:
m = 0
ds = ['xrel','yrel','zrel']
X = envs
size = len(X)*3
x1 = X[int(math.floor(m/3))]
d1 = ds[m%3]
sig = 1
ls = 1
kernel = two_body

time1 = time.time()
get_cov_row(x1, d1, m, size, X, sig, ls, kernel)
time2 = time.time()
print(time2-time1)

0.05656027793884277


In [8]:
# cov row: serial version
time1 = time.time()
ds = ['xrel','yrel','zrel']
ser_covs = []
for n in range(m,size):
    x2 = X[int(math.floor(n/3))]
    d2 = ds[n%3]

    # calculate kernel
    ser_covs.append(kernel(x1, x2, d1, d2, sig, ls))
time2 = time.time()
print(time2-time1)

20.803579807281494


In [15]:
# cov row: parallel version
pool = mp.Pool(processes=2)
time1 = time.time()
ds = ['xrel','yrel','zrel']
procs = []
par_covs = []
for n in range(m,size):
    x2 = X[int(math.floor(n/3))]
    d2 = ds[n%3]

    # calculate kernel
    procs.append(pool.apply_async(kernel, \
            args=(x1, x2, d1, d2, sig, ls)))

for n in range(len(procs)):
    par_covs.append(procs[n].get())
pool.close()
time2 = time.time()
print(time2-time1)

11.7795729637146


Test covariance matrix function.

In [4]:
# serial version
X = envs
sig = 1
ls = 1
kernel = two_body
noise=1
time1 = time.time()
get_K(X,sig,ls,noise,kernel)
time2 = time.time()
print(time2-time1)

3.1088719367980957


In [6]:
# parallel version
X = envs
sig = 1
ls = 1
kernel = two_body
pool = mp.Pool(processes=2)
noise=1
time1 = time.time()
get_K_par(X,sig,ls,noise,pool,kernel)
time2 = time.time()
print(time2-time1)

40.702632904052734


Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/jonpvandermause/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/jonpvandermause/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/jonpvandermause/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jonpvandermause/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/jonpvandermause/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jonpvandermause/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/jonpvandermause/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    

Develop parallel covariance function that distributes each comparison to a new processor.

In [7]:
# serial version
# set inputs
X = envs
sig = 1
ls = 1
kernel = two_body
# pool = mp.Pool(processes=2)
noise=1

time1 = time.time()
# begin function
ds = ['xrel','yrel','zrel']

# initialize matrix
size = len(X)*3
K = np.zeros([size, size])
results=[]

# calculate elements
for m in range(size):
    x1 = X[int(math.floor(m/3))]
    d1 = ds[m%3]
    for n in range(m,size):
        x2 = X[int(math.floor(n/3))]
        d2 = ds[n%3]

        # calculate kernel
#         results.append(pool.apply_async(kernel, \
#             args=(x1, x2, d1, d2, sig, ls)))

        results.append(kernel(x1, x2, d1, d2, sig, ls))
    
coll = []
count = 0
for m in range(size):
    for n in range(m,size):
        K[m,n]=results[count]
        K[n,m]=results[count]
        count+=1
        
#         coll.append(results[m].get())

time2 = time.time()
print(time2-time1)

# # perform cholesky decomposition
# L = np.linalg.cholesky(K+noise**2*np.eye(size))

NameError: name 'envs' is not defined

In [4]:
# parallel version
# set inputs
X = envs
sig = 1
ls = 1
kernel = two_body
pool = mp.Pool(processes=2)
noise=1

time1 = time.time()
# begin function
ds = ['xrel','yrel','zrel']

# initialize matrix
size = len(X)*3
K = np.zeros([size, size])
results=[]

# calculate elements
for m in range(size):
    x1 = X[int(math.floor(m/3))]
    d1 = ds[m%3]
    for n in range(m,size):
        x2 = X[int(math.floor(n/3))]
        d2 = ds[n%3]

        # calculate kernel
        results.append(pool.apply_async(kernel, \
            args=(x1, x2, d1, d2, sig, ls)))

count = 0
for m in range(size):
    for n in range(m,size):
        val = results[count].get()
        K[m,n]=val
        K[n,m]=val
        count+=1


pool.close()
time2 = time.time()
print(time2-time1)

# # perform cholesky decomposition
# L = np.linalg.cholesky(K+noise**2*np.eye(size))

NameError: name 'envs' is not defined

Develop smarter parallelization scheme. Split covariance matrix calculation into lists of environment comparisons, with one list assigned to each processor.

First, develop a function that creates assignment lists.

In [12]:
def get_assignment(X,batches):
    ds = ['xrel','yrel','zrel']
    size = len(X)*3
    tot_comps = (size+1)*size/2
    switch = math.ceil(tot_comps/batches)
    batch_count = 0
    assign = []

    # calculate elements
    for m in range(size):
        x1_ind = int(math.floor(m/3))
        d1 = ds[m%3]
        for n in range(m,size):
            x2_ind = int(math.floor(n/3))
            d2 = ds[n%3]

            # when batch limit is reached, reset counter
            if batch_count == switch:
                batch_count = 0

            # if counter is zero, start a new list
            if batch_count==0:
                assign.append([[x1_ind,d1,x2_ind,d2]])

            # otherwise, add comparison to last list
            else:
                assign[len(assign)-1].append([x1_ind,d1,x2_ind,d2])

            # increment counter
            batch_count+=1

    return assign

X = envs
batches = 2
assign = get_assignment(X,batches)

Given an assignment (i.e. a list of environment pairs), compute the corresponding list of covariances.

In [13]:
def get_cov_list(pair_list, X, kernel, sig, ls):
    res_store = []
    for n in range(len(pair_list)):
        pair = pair_list[n]
        x1 = X[pair[0]]
        d1 = pair[1]
        x2 = X[pair[2]]
        d2 = pair[3]
        res_store.append(kernel(x1, x2, d1, d2, sig, ls))
    return res_store

# pair_list = assign[0]
# kernel = two_body
# sig = 1
# ls = 1

# time1 = time.time()
# test = get_cov_list(pair_list, X, kernel, sig, ls)
# time2 = time.time()
# print(time2-time1)

Given a list of assignments, assign each assignment to a processor and collect the results.

In [47]:
def get_K_pool(X, batches, kernel, sig, ls, noise):
    # get assignment
    assign = get_assignment(X, batches)
    
    # create pool of processors
    procs = len(assign)
    pool = mp.Pool(processes = procs)

    # assign to processors
    res = []
    for n in range(procs):
        # set pair list
        pair_list = assign[n]

        # assign pair list to processor
        res.append(pool.apply_async(get_cov_list,\
            args = (pair_list, X, kernel, sig, ls)))

    # collect covariance lists
    results = []
    for n in range(procs):
        time1=time.time()
        results.append(res[n].get())
        time2=time.time()
        print(time2-time1)
        
    # create K matrix from covariance lists
    size = len(X)*3
    K = np.zeros([size, size])
    counter = 0
    cov_list = 0
    for m in range(size):
        for n in range(m,size):
            if counter < len(results[cov_list]):
                ent = results[cov_list][counter]
            else:
                counter = 0
                cov_list +=1
                ent = results[cov_list][counter]
            
            counter+=1
            K[m,n]=ent
            K[n,m]=ent

    # close the pool
    pool.close()
    
    # perform cholesky decomposition
    L = np.linalg.cholesky(K+noise**2*np.eye(size))
    
    return K, L

Use manager to make the list of environment dictionaries public.

In [75]:
X_shared[0]

FileNotFoundError: [Errno 2] No such file or directory

In [51]:
def get_K_man(X, batches, kernel, sig, ls, noise):
    # get assignment
    assign = get_assignment(X, batches)
    
    # create manager
    manager = mp.Manager()
    X_shared = manager.list(X)
    
    # create pool of processors
    procs = len(assign)
    pool = mp.Pool(processes = procs)

    # assign to processors
    res = []
    for n in range(procs):
        # set pair list
        pair_list = assign[n]

        # assign pair list to processor
        res.append(pool.apply_async(get_cov_list,\
            args = (pair_list, X_shared, kernel, sig, ls)))

#     # close and join the pool
#     pool.close()
#     pool.join()
    
    # collect covariance lists
    results = []
    for n in range(procs):
        time1=time.time()
        results.append(res[n].get())
        time2=time.time()
        print(time2-time1)
        
    # create K matrix from covariance lists
    size = len(X)*3
    K = np.zeros([size, size])
    counter = 0
    cov_list = 0
    for m in range(size):
        for n in range(m,size):
            if counter < len(results[cov_list]):
                ent = results[cov_list][counter]
            else:
                counter = 0
                cov_list +=1
                ent = results[cov_list][counter]
            
            counter+=1
            K[m,n]=ent
            K[n,m]=ent

    # close the pool
    pool.close()
    
    # perform cholesky decomposition
    L = np.linalg.cholesky(K+noise**2*np.eye(size))
    
    return K, L

In [50]:
manager = mp.Manager()
X_shared = manager.list(X)

In [2]:
# map test
def test_func(x):
    return x

In [6]:
pool = mp.Pool(2)
args = [[1,2],[3,4]]
test = pool.map(test_func, args)
pool.close()

In [5]:
test

[[1, 2], [3, 4]]

In [7]:
mp.cpu_count()

4

In [8]:
pool1 = mp.Pool(2)

In [9]:
pool2 = mp.Pool(2)

In [10]:
pool3 = mp.Pool(2)

In [11]:
import mpi4py

ModuleNotFoundError: No module named 'mpi4py'